# Generating offspring photos with ComfyUI

## 1. Set Up

In [ ]:
!git clone https://github.com/comfyanonymous/ComfyUI.git

In [ ]:
!pip install -r requirements.txt

In [23]:
import sys
import os
from pathlib import Path

In [24]:
# Define my specific ComfyUI path
COMFY_PATH = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI")

# Verify the path exists
if not COMFY_PATH.exists():
    raise Exception(f"ComfyUI path not found at {COMFY_PATH}")
else:
    print(f"ComfyUI directory found at {COMFY_PATH}")

# Add ComfyUI to system path
if str(COMFY_PATH) not in sys.path:
    sys.path.append(str(COMFY_PATH))
    print("Added ComfyUI to system path")

ComfyUI directory found at /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI


In [25]:
import requests
from tqdm import tqdm

In [26]:
def download_model(url, save_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    save_path = Path(save_path)
    save_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(save_path, 'wb') as f, tqdm(
        desc=save_path.name,
        total=total_size,
        unit='iB',
        unit_scale=True
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            pbar.update(size)

In [27]:
# Create model directories
models_path = COMFY_PATH / "models"
checkpoints_path = models_path / "checkpoints"
controlnet_path = models_path / "controlnet"

In [28]:
# Create directories and print status
for path in [models_path, checkpoints_path, controlnet_path]:
    path.mkdir(parents=True, exist_ok=True)
    print(f"Created/verified directory: {path}")

Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/models
Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/models/checkpoints
Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/models/controlnet


## Start the ComfyUI server

In [29]:
import subprocess
import webbrowser
from time import sleep

In [30]:
# Start ComfyUI server
server_process = subprocess.Popen(
    ['python', str(COMFY_PATH / 'main.py'), '--listen', '0.0.0.0', '--port', '8188'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait a moment for the server to start
sleep(5)

# Open the UI in default browser
webbrowser.open('http://localhost:8188')

print("ComfyUI server started! The interface should open in your browser.")
print("Keep this notebook running while you use ComfyUI.")

ComfyUI server started! The interface should open in your browser.
Keep this notebook running while you use ComfyUI.


## 3. Setting up Paths

In [31]:
import json
import csv
from datetime import datetime
import shutil
from pathlib import Path

In [32]:
def setup_donor_folders(donor_filename):
    """
    Creates the folder structure for a donor's generations
    """
    donor_stem = Path(donor_filename).stem
    donor_dir = PATHS["output"] / "donor_logs" / donor_stem / "v1"
    donor_dir.mkdir(parents=True, exist_ok=True)
    return donor_dir

def create_generation_metadata(donor_filename, seeds, cfg=7.0, steps=25, parameters=None):
    """Creates metadata for a single donor's generation session"""
    return {
        "donor_image": donor_filename,
        "timestamp": datetime.now().isoformat(),
        "generation_params": parameters if parameters else {
            "cfg": cfg,
            "steps": steps,
            "sampler": "euler",
        },
        "seeds": seeds,
        "output_files": [
            f"{Path(donor_filename).stem}_baby_{i+1}.png" 
            for i in range(len(seeds))
        ]
    }

def update_main_logs(metadata, csv_path, json_path):
    """
    Updates both CSV and JSON logs with new generation data
    """
    # Update JSON log
    if json_path.exists():
        with open(json_path, 'r') as f:
            all_logs = json.load(f)
    else:
        all_logs = []
    
    all_logs.append(metadata)
    with open(json_path, 'w') as f:
        json.dump(all_logs, f, indent=2)

    # Update CSV log
    csv_exists = csv_path.exists()
    with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        if not csv_exists:
            writer.writerow([
                'donor_image', 'timestamp', 'cfg', 
                'steps', 'sampler', 'seeds', 'output_files'
            ])
        writer.writerow([
            metadata['donor_image'],
            metadata['timestamp'],
            metadata['generation_params']['cfg'],
            metadata['generation_params']['steps'],
            metadata['generation_params']['sampler'],
            ','.join(map(str, metadata['seeds'])),
            ','.join(metadata['output_files'])
        ])

In [33]:
# Create input directory for my photo
input_path = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input")
input_path.mkdir(parents=True, exist_ok=True)

# Paths dictionary
PATHS = {
    "donors": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files"),
    "output": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings"),
    "my_photo": input_path / "cindy.jpg" 
}

In [34]:
# Verify paths
for name, path in PATHS.items():
    if path.exists():
        print(f"✓ Found {name} directory: {path}")
    else:
        if name == "my_photo":
            print(f"\nPlease save your headshot photo as: {path}")
            print("Once saved, run this cell again to verify.")
        elif name == "output":
            path.mkdir(parents=True, exist_ok=True)
            print(f"Created output directory: {path}")
        else:
            print(f"❌ ERROR: {name} directory not found: {path}")

✓ Found donors directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files
✓ Found output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings
✓ Found my_photo directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/cindy.jpg


In [35]:
import shutil
from pathlib import Path

In [36]:
# Paths cleanup (to comply with ComfyUI's output structure)
PATHS = {
    "donors": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files"),
    "output": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings"),
    "my_photo": input_path / "cindy.jpg" 
}

# 1. Clean up: Remove any previously created symlinks or directories
project_outputs = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/thesis_outputs")
if project_outputs.exists():
    if project_outputs.is_symlink():
        project_outputs.unlink()
    elif project_outputs.is_dir():
        shutil.rmtree(project_outputs)

# 2. Set up fresh directory structure in ComfyUI's output
comfy_output = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output")
donor_output = comfy_output / "donor_logs" / "aug_gpt_4_gem_wo" / "v1"
donor_output.mkdir(parents=True, exist_ok=True)

print("Cleaned up previous setup and created fresh directory structure.")
print(f"ComfyUI output directory: {comfy_output}")
print(f"Donor output directory: {donor_output}")

Cleaned up previous setup and created fresh directory structure.
ComfyUI output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output
Donor output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/donor_logs/aug_gpt_4_gem_wo/v1


## 6. Batch Processing
Testing with 5 donors first to ensure everything works properly

In [37]:
# Import and Setup
import json
import time
from pathlib import Path
import shutil
import random

In [44]:
# Update paths
COMFY_PATH = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI")
COMFY_OUTPUT = COMFY_PATH / "output"
DONORS_PATH = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files")
MY_PHOTO = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/cindy.jpg")
BASE_WORKFLOW_PATH = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/user/default/workflows/base_workflow.json")

In [ ]:
test_donors = sorted(list(DONORS_PATH.glob("*.jpeg")))[:5]
print("\nWill process these donors:")
for i, donor in enumerate(test_donors, 1):
    print(f"{i}. {donor.name}")


Will process these donors:
1. aug_cla_2_gem_wo.jpeg
2. aug_cla_3_gem_wo.jpeg
3. aug_cla_4_gem_wo.jpeg
4. aug_cla_5_gem_wo.jpeg
5. aug_cla_6_gem_demo.jpeg


In [40]:
# Utility Functions
def verify_saved_image(output_path: Path, donor_id: str, variation: int) -> bool:
    """Verify that image was saved and is valid"""
    expected_file = list(output_path.glob(f"{donor_id}_baby_{variation}*.png"))
    
    if not expected_file:
        print(f"❌ Warning: No output file found for {donor_id} variation {variation}")
        return False
        
    if expected_file[0].stat().st_size == 0:
        print(f"❌ Warning: Empty file generated for {donor_id} variation {variation}")
        return False
        
    print(f"✓ Successfully saved: {expected_file[0].name}")
    return True

def generate_random_seeds(count=3):
    """Generate list of random seeds for multiple generations"""
    return [random.randint(1, 2**31 - 1) for _ in range(count)]


In [53]:
# Generate random values for steps and cfg
random_steps = random.randint(10, 100)  # Random value between 10 and 100
random_cfg = round(random.uniform(0.5, 10.0), 2)  # Random float between 0.5 and 10, rounded to 2 decimal places
random_seed = random.randint(1, 2**31 - 1)  # or use a predefined list of seeds
random_denoise = round(random.uniform(0.0, 1.0), 2)  # rounded to 2 decimal places

In [55]:
complete_workflow = {
    "version": 1.0,
    "state": {},
    "nodes": [
        {
            "id": 1,
            "type": "LoadImage",
            "pos": [50, 200],
            "size": [315, 98],
            "flags": {},
            "order": 0,
            "mode": 0,
            "properties": {"Node name for S&R": "LoadImage"},
            "inputs": [],
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [5],
                    "slot_index": 0
                },
                {
                    "name": "MASK",
                    "type": "MASK",
                    "links": [],
                    "slot_index": 1
                }
            ],
            "widgets_values": [str(MY_PHOTO)]
        },
        {
            "id": 2,
            "type": "LoadImage",
            "pos": [50, 400],
            "size": [315, 98],
            "flags": {},
            "order": 1,
            "mode": 0,
            "properties": {"Node name for S&R": "LoadImage"},
            "inputs": [],
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [6],
                    "slot_index": 0
                },
                {
                    "name": "MASK",
                    "type": "MASK",
                    "links": [],
                    "slot_index": 1
                }
            ],
            "widgets_values": [str(test_donors)]
        },
        {
            "id": 3,
            "type": "CheckpointLoaderSimple",
            "pos": [50, 0],
            "size": [300, 100],
            "flags": {},
            "order": 2,
            "mode": 0,
            "properties": {"Node name for S&R": "CheckpointLoaderSimple"},
            "inputs": [],
            "outputs": [
                {"name": "MODEL", "type": "MODEL", "links": [7], "slot_index": 0},
                {"name": "CLIP", "type": "CLIP", "links": [8, 15], "slot_index": 1},
                {"name": "VAE", "type": "VAE", "links": [9], "slot_index": 2}
            ],
            "widgets_values": ["sd_xl_turbo_1.0.safetensors"]
        },
        {
            "id": 4,
            "type": "CLIPTextEncode",
            "pos": [400, 0],
            "size": [400, 100],
            "flags": {},
            "order": 3,
            "mode": 0,
            "properties": {"Node name for S&R": "CLIPTextEncode"},
            "inputs": [
                {"name": "clip", "type": "CLIP", "link": 8, "slot_index": 0}
            ],
            "outputs": [
                {"name": "CONDITIONING", "type": "CONDITIONING", "links": [10], "slot_index": 0}
            ],
            "widgets_values": ["portrait of a 12-18 month old baby with distinctive facial features, natural pose, candid expression, detailed facial features, high quality photograph, photorealistic"]
        },
        {
            "id": 5,
            "type": "KSampler",
            "pos": [800, 200],
            "size": [400, 400],
            "flags": {},
            "order": 4,
            "mode": 0,
            "properties": {"Node name for S&R": "KSampler"},
            "inputs": [
                {"name": "model", "type": "MODEL", "link": 7, "slot_index": 0},
                {"name": "positive", "type": "CONDITIONING", "link": 10, "slot_index": 1},
                {"name": "negative", "type": "CONDITIONING", "link": 11, "slot_index": 2},
                {"name": "latent_image", "type": "LATENT", "link": 12, "slot_index": 3}
            ],
            "outputs": [
                {"name": "LATENT", "type": "LATENT", "links": [13], "slot_index": 0}
            ],
            "widgets_values": [
                random_seed,          # seed
                "randomize",       # control_after_generate
                random_steps,               # steps
                random_cfg,              # cfg
                "euler",   # sampler_name
                "karras",         # scheduler
                random_denoise              # denoise
            ]
        },
        {
            "id": 8,
            "type": "EmptyLatentImage",
            "pos": [600, 300],
            "size": [315, 98],
            "flags": {},
            "order": 3,
            "mode": 0,
            "properties": {"Node name for S&R": "EmptyLatentImage"},
            "inputs": [],
            "outputs": [
                {"name": "LATENT", "type": "LATENT", "links": [12], "slot_index": 0}
            ],
            "widgets_values": [512, 512, 1]
        },
        {
            "id": 9,
            "type": "CLIPTextEncode",
            "pos": [400, 150],
            "size": [400, 100],
            "flags": {},
            "order": 3,
            "mode": 0,
            "properties": {"Node name for S&R": "CLIPTextEncode"},
            "inputs": [
                {"name": "clip", "type": "CLIP", "link": 15, "slot_index": 0}
            ],
            "outputs": [
                {"name": "CONDITIONING", "type": "CONDITIONING", "links": [11], "slot_index": 0}
            ],
            "widgets_values": ["distorted, deformed, ugly, mutation, blurry, grainy, noisy, oversaturated, overprocessed, artificial, harsh lighting, unrealistic colors, plastic skin, extreme contrast, overexposed, surreal, watercolor, painting, illustration, artificial HDR, oversharpened"]
        },
        {
            "id": 6,
            "type": "VAEDecode",
            "pos": [1200, 200],
            "size": [200, 100],
            "flags": {},
            "order": 5,
            "mode": 0,
            "properties": {"Node name for S&R": "VAEDecode"},
            "inputs": [
                {"name": "samples", "type": "LATENT", "link": 13, "slot_index": 0},
                {"name": "vae", "type": "VAE", "link": 9, "slot_index": 1}
            ],
            "outputs": [
                {"name": "IMAGE", "type": "IMAGE", "links": [14], "slot_index": 0}
            ]
        },
        {
            "id": 7,
            "type": "SaveImage",
            "pos": [1400, 200],
            "size": [400, 100],
            "flags": {},
            "order": 8,
            "mode": 0,
            "inputs": [{"name": "images", "type": "IMAGE", "link": 14, "slot_index": 0}],
            "outputs": [],
            "properties": {"Node name for S&R": "SaveImage"},
            "widgets_values": ["aug_cla_2_gem_wo_baby_1"]
        }
    ],
    "links": [
        {"id": 5, "origin_id": 1, "origin_slot": 0, "target_id": 5, "target_slot": 0, "type": "IMAGE"},
        {"id": 6, "origin_id": 2, "origin_slot": 0, "target_id": 5, "target_slot": 1, "type": "IMAGE"},
        {"id": 7, "origin_id": 3, "origin_slot": 0, "target_id": 5, "target_slot": 0, "type": "MODEL"},
        {"id": 8, "origin_id": 3, "origin_slot": 1, "target_id": 4, "target_slot": 0, "type": "CLIP"},
        {"id": 9, "origin_id": 3, "origin_slot": 2, "target_id": 6, "target_slot": 1, "type": "VAE"},
        {"id": 10, "origin_id": 4, "origin_slot": 0, "target_id": 5, "target_slot": 1, "type": "CONDITIONING"},
        {"id": 11, "origin_id": 9, "origin_slot": 0, "target_id": 5, "target_slot": 2, "type": "CONDITIONING"},
        {"id": 12, "origin_id": 8, "origin_slot": 0, "target_id": 5, "target_slot": 3, "type": "LATENT"},
        {"id": 13, "origin_id": 5, "origin_slot": 0, "target_id": 6, "target_slot": 0, "type": "LATENT"},
        {"id": 14, "origin_id": 6, "origin_slot": 0, "target_id": 7, "target_slot": 0, "type": "IMAGE"},
        {"id": 15, "origin_id": 3, "origin_slot": 1, "target_id": 9, "target_slot": 0, "type": "CLIP"}
    ]
}

In [ ]:
# def modify_workflow_for_donor(workflow: dict, donor_path: str, seed: int, variation: int) -> dict:
#     """Modify workflow for specific donor and variation"""
#     modified = workflow.copy()
    
#     # Create random parameters once
#     parameters = {
#         'seed': random.randint(1, 2**31 - 1),
#         'steps': random.randint(25, 40),
#         'cfg': round(random.uniform(6.0, 11.0), 1),
#         'sampler': 'euler',
#         'scheduler': random.choice(['normal', 'karras']),
#         'denoise': round(random.uniform(0.7, 0.9), 2)
#     }
    
#     # Add parameters to the workflow dict
#     modified['generation_parameters'] = parameters
    
#     # Update nodes
#     for node in modified['nodes']:
#         if node['type'] == 'LoadImage':
#             if node['id'] == 1:  # Your photo
#                 node['widgets_values'] = [str(MY_PHOTO)]
#             elif node['id'] == 2:  # Donor photo
#                 node['widgets_values'] = [str(donor_path)]
        
#         # Update KSampler with random parameters
#         elif node['type'] == 'KSampler':
#             node['widgets_values'] = [
#                 parameters['seed'],
#                 "randomize",    
#                 parameters['steps'],
#                 parameters['cfg'],
#                 parameters['sampler'],
#                 parameters['scheduler'],
#                 parameters['denoise']
#             ]
        
#         # Update SaveImage filename
#         elif node['type'] == 'SaveImage':
#             donor_id = Path(donor_path).stem
#             node['widgets_values'] = [f"{donor_id}_baby_{variation}"]
    
#     return modified

In [56]:
# Load and modify base workflow
print(f"Loading base workflow from: {BASE_WORKFLOW_PATH}")
if not BASE_WORKFLOW_PATH.exists():
    raise FileNotFoundError(f"Base workflow not found at: {BASE_WORKFLOW_PATH}")

with open(BASE_WORKFLOW_PATH, 'r') as f:
    base_workflow = json.load(f)
print("Successfully loaded base workflow")



Loading base workflow from: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/user/default/workflows/base_workflow.json
Successfully loaded base workflow


### Test Batch Processing Setup

In [57]:
# Get first 5 donors
test_donors = sorted(list(DONORS_PATH.glob("*.jpeg")))[:5]

print("\nWill process these donors:")
for i, donor in enumerate(test_donors, 1):
    print(f"{i}. {donor.name}")



Will process these donors:
1. aug_cla_2_gem_wo.jpeg
2. aug_cla_3_gem_wo.jpeg
3. aug_cla_4_gem_wo.jpeg
4. aug_cla_5_gem_wo.jpeg
5. aug_cla_6_gem_demo.jpeg


### Process Test Batch

In [58]:
def process_test_batch():
    total_donors = len(test_donors)
    
    for donor_idx, donor_path in enumerate(test_donors, 1):
        print(f"\n{'='*50}")
        print(f"Processing donor {donor_idx}/{total_donors}: {donor_path.name}")
        
        # Generate 3 variations for this donor
        seeds = generate_random_seeds(3)
        
        for variation in range(1, 4):
            print(f"\n>> Starting variation {variation}/3")
            print("Generating random parameters...")
            
            # Modify workflow for this variation
            workflow = modify_workflow_for_donor(
                base_workflow,
                str(donor_path),
                seeds[variation-1],
                variation
            )
            
            # Print the parameters being used - ADDED MORE VISIBLE FORMATTING
            params = workflow['generation_parameters']
            print("\n==== GENERATION PARAMETERS ====")
            print(f"Steps: {params['steps']}")
            print(f"CFG: {params['cfg']}")
            print(f"Sampler: {params['sampler']}")
            print(f"Scheduler: {params['scheduler']}")
            print(f"Denoise: {params['denoise']}")
            print(f"Seed: {params['seed']}")
            print("=============================\n")
            
            # Save workflow for this variation
            workflow_path = COMFY_OUTPUT / "workflows" / f"{donor_path.stem}_v{variation}.json"
            workflow_path.parent.mkdir(parents=True, exist_ok=True)
            
            with open(workflow_path, 'w') as f:
                json.dump(workflow, f, indent=2)
            
            print(f"✓ Saved workflow to: {workflow_path}")
            print(f"\n⚠️  ACTION NEEDED:")
            print(f"1. Load the workflow in ComfyUI")
            print(f"2. Click 'Queue' to generate")
            print(f"3. Wait for generation to complete")
            
            input("\nPress Enter after the image has been generated...")
            
            # Verify saved image
            verify_saved_image(COMFY_OUTPUT, donor_path.stem, variation)
            
            # Brief pause between generations
            time.sleep(2)
        
        print(f"\n✓ Completed all variations for {donor_path.name}")
    
    print("\n🎉 Batch processing complete!")

# Then let's add a confirmation before starting:
print("Starting test batch with randomized parameters for 5 donors...")
proceed = input("Press Enter to continue or Ctrl+C to cancel...")
process_test_batch()

Starting test batch with randomized parameters for 5 donors...

Processing donor 1/5: aug_cla_2_gem_wo.jpeg

>> Starting variation 1/3
Generating random parameters...

==== GENERATION PARAMETERS ====
Steps: 38
CFG: 8.0
Sampler: euler
Scheduler: normal
Denoise: 0.7
Seed: 2131144659

✓ Saved workflow to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/aug_cla_2_gem_wo_v1.json

⚠️  ACTION NEEDED:
1. Load the workflow in ComfyUI
2. Click 'Queue' to generate
3. Wait for generation to complete
✓ Successfully saved: aug_cla_2_gem_wo_baby_1_00001_.png

>> Starting variation 2/3
Generating random parameters...

==== GENERATION PARAMETERS ====
Steps: 31
CFG: 9.9
Sampler: euler
Scheduler: normal
Denoise: 0.73
Seed: 2063132830

✓ Saved workflow to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/aug_cla_2_gem_wo_v2.json

⚠️  ACTION NEEDED:
1. Load the workflow in ComfyUI
2. Click 'Queue' to generate
3. 